# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 21 2022 10:18AM,253578,16,TASA-284995,TASA USA Inc.,Released
1,Dec 21 2022 10:16AM,253577,20,7370188,"Brookfield Pharmaceuticals, LLC",Released
2,Dec 21 2022 10:00AM,253575,16,SSF_C1W52,Sartorius Stedim Filters Inc.,Released
3,Dec 21 2022 9:49AM,253574,19,STR12212022-1,ACG North America LLC,Released
4,Dec 21 2022 9:49AM,253574,19,STR12212022-2,ACG North America LLC,Released
5,Dec 21 2022 9:40AM,253573,10,0086262876,ISDIN Corporation,Released
6,Dec 21 2022 9:40AM,253572,10,0086263377,ISDIN Corporation,Released
7,Dec 21 2022 9:40AM,253572,10,0086263379,ISDIN Corporation,Released
8,Dec 21 2022 9:25AM,253570,10,0086262768,ISDIN Corporation,Released
9,Dec 21 2022 9:25AM,253570,10,0086262849,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,253573,Released,1
24,253574,Released,2
25,253575,Released,1
26,253577,Released,1
27,253578,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253573,NaN,1.0
253574,NaN,2.0
253575,NaN,1.0
253577,NaN,1.0
253578,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253488,0.0,1.0
253494,0.0,42.0
253495,0.0,19.0
253497,13.0,2.0
253502,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253488,0,1
253494,0,42
253495,0,19
253497,13,2
253502,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253488,0,1
1,253494,0,42
2,253495,0,19
3,253497,13,2
4,253502,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253488,,1
1,253494,,42
2,253495,,19
3,253497,13,2
4,253502,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 21 2022 10:18AM,253578,16,TASA USA Inc.
1,Dec 21 2022 10:16AM,253577,20,"Brookfield Pharmaceuticals, LLC"
2,Dec 21 2022 10:00AM,253575,16,Sartorius Stedim Filters Inc.
3,Dec 21 2022 9:49AM,253574,19,ACG North America LLC
5,Dec 21 2022 9:40AM,253573,10,ISDIN Corporation
6,Dec 21 2022 9:40AM,253572,10,ISDIN Corporation
8,Dec 21 2022 9:25AM,253570,10,ISDIN Corporation
31,Dec 21 2022 9:06AM,253565,10,ISDIN Corporation
76,Dec 21 2022 9:04AM,253569,10,"Methapharm, Inc."
79,Dec 21 2022 9:04AM,253567,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 21 2022 10:18AM,253578,16,TASA USA Inc.,,1
1,Dec 21 2022 10:16AM,253577,20,"Brookfield Pharmaceuticals, LLC",,1
2,Dec 21 2022 10:00AM,253575,16,Sartorius Stedim Filters Inc.,,1
3,Dec 21 2022 9:49AM,253574,19,ACG North America LLC,,2
4,Dec 21 2022 9:40AM,253573,10,ISDIN Corporation,,1
5,Dec 21 2022 9:40AM,253572,10,ISDIN Corporation,,2
6,Dec 21 2022 9:25AM,253570,10,ISDIN Corporation,,23
7,Dec 21 2022 9:06AM,253565,10,ISDIN Corporation,,45
8,Dec 21 2022 9:04AM,253569,10,"Methapharm, Inc.",,3
9,Dec 21 2022 9:04AM,253567,10,"Methapharm, Inc.",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 21 2022 10:18AM,253578,16,TASA USA Inc.,1,
1,Dec 21 2022 10:16AM,253577,20,"Brookfield Pharmaceuticals, LLC",1,
2,Dec 21 2022 10:00AM,253575,16,Sartorius Stedim Filters Inc.,1,
3,Dec 21 2022 9:49AM,253574,19,ACG North America LLC,2,
4,Dec 21 2022 9:40AM,253573,10,ISDIN Corporation,1,
5,Dec 21 2022 9:40AM,253572,10,ISDIN Corporation,2,
6,Dec 21 2022 9:25AM,253570,10,ISDIN Corporation,23,
7,Dec 21 2022 9:06AM,253565,10,ISDIN Corporation,45,
8,Dec 21 2022 9:04AM,253569,10,"Methapharm, Inc.",3,
9,Dec 21 2022 9:04AM,253567,10,"Methapharm, Inc.",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 21 2022 10:18AM,253578,16,TASA USA Inc.,1,
1,Dec 21 2022 10:16AM,253577,20,"Brookfield Pharmaceuticals, LLC",1,
2,Dec 21 2022 10:00AM,253575,16,Sartorius Stedim Filters Inc.,1,
3,Dec 21 2022 9:49AM,253574,19,ACG North America LLC,2,
4,Dec 21 2022 9:40AM,253573,10,ISDIN Corporation,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 21 2022 10:18AM,253578,16,TASA USA Inc.,1,NaN
1,Dec 21 2022 10:16AM,253577,20,"Brookfield Pharmaceuticals, LLC",1,NaN
2,Dec 21 2022 10:00AM,253575,16,Sartorius Stedim Filters Inc.,1,NaN
3,Dec 21 2022 9:49AM,253574,19,ACG North America LLC,2,NaN
4,Dec 21 2022 9:40AM,253573,10,ISDIN Corporation,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 21 2022 10:18AM,253578,16,TASA USA Inc.,1,0.0
1,Dec 21 2022 10:16AM,253577,20,"Brookfield Pharmaceuticals, LLC",1,0.0
2,Dec 21 2022 10:00AM,253575,16,Sartorius Stedim Filters Inc.,1,0.0
3,Dec 21 2022 9:49AM,253574,19,ACG North America LLC,2,0.0
4,Dec 21 2022 9:40AM,253573,10,ISDIN Corporation,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5070868,118,13.0
15,506989,61,0.0
16,507153,2,0.0
19,760698,90,1.0
20,253577,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5070868,118,13.0
1,15,506989,61,0.0
2,16,507153,2,0.0
3,19,760698,90,1.0
4,20,253577,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,118,13.0
1,15,61,0.0
2,16,2,0.0
3,19,90,1.0
4,20,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,118.0
1,15,Released,61.0
2,16,Released,2.0
3,19,Released,90.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Executing,13.0,0.0,0.0,1.0,0.0
Released,118.0,61.0,2.0,90.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Executing,13.0,0.0,0.0,1.0,0.0
1,Released,118.0,61.0,2.0,90.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Executing,13.0,0.0,0.0,1.0,0.0
1,Released,118.0,61.0,2.0,90.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()